For this assignment I will be using the Pytesseract library to convert the given PDF into images, and then use OCR to recognize the content of the image and store it in a text file.
* First the PDF will be converted into image files. Each page of the PDF will be stored as a seperate image.
* Then using Pytesseract, the content of the image will be recognized and stored in a text file.

Then using NLTK and regex, the required values will be extracted.

In [1]:
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 
import time
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [2]:
timestr = time.strftime("%Y%m%d%H%M%S")
pages = convert_from_path("file_3.pdf", 500)
image_counter = 1
# Iterate through all the pages stored above and save them a JPEG files
for page in pages: 
  
    filename = "page_"+str(image_counter)+".jpg"
    page.save(filename, 'JPEG') 
    image_counter = image_counter + 1
  
 
# Recognizing text from the images using OCR and storing in a timestamped text file
filelimit = image_counter-1
outfile = "out_text_"+timestr+".txt"
f = open(outfile, "a") 
for i in range(1, filelimit + 1): 
    filename = "page_"+str(i)+".jpg"      
    # Recognize the text as string in image using pytesserct 
    text = str(((pytesseract.image_to_string(Image.open(filename))))) 
    f.write(text) 
f.close() 

# Read the text file created and store the content in a variable
f = open(outfile, "r") 
text = f.read()
f.close()

In [3]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
stop = stopwords.words('english')
def ie_preprocess(document):
    document = ' '.join([i for i in document.split() if i not in stop])
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    return sentences

def extract_names(document):
    names = []
    sentences = ie_preprocess(document)
    for tagged_sentence in sentences:
        for chunk in nltk.ne_chunk(tagged_sentence):
            if type(chunk) == nltk.tree.Tree:
                if chunk.label() == 'PERSON':
                    names.append(' '.join([c[0] for c in chunk]))
    return names


[nltk_data] Downloading package stopwords to C:\Users\Aastha
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Aastha
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Aastha Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to C:\Users\Aastha
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to C:\Users\Aastha
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [4]:
# Use the extract names method from nltk Library to extract all the possible names in the text file
names = list(set(extract_names(text)))
  
accused = ''
#From the list of names generated from extract_names method find the name of the accused
for name in names:
    if len(re.findall(r'accused\s*'+name,text)) != 0:
        accused = name

# Finding the address of the accused
a= re.findall(r'r/o:?(.*)\n\n(?:5.|par)',text,re.S | re.I)
address = a[0].replace('\n',', ')

# Finding whether the accused is convicted or acquited
final_decision = ''
if re.search("acquited",text, re.I):
    final_decision = "Acquited"
elif re.search("convicted",text, re.I):
    final_decision = "Convited"  

# Date of Decisopn    
date = re.findall(r'(\d+/\d+/\d+)',text)[-1]

# Sections under which the accused was charged
l = re.findall(r'sec.-?\w*-?\s*(\d*)',text, re.I)
lt = ' '.join(map(str, l)) 
sections = list(set(re.sub(r'sec.\w*','',lt).split()))
                    
                    
print("Name of the accused is: ",accused)
print("Address of the accused:",address)
print("Sections under which the accused was charged: ",sections)
print("Whether is accused is convicted or acquited: ",final_decision)
print("Date of verdict: ",date)

Name of the accused is:  Suresh Punjaram Sable
Address of the accused:  Valadgaon, Tal. Aurangabad
Sections under which the accused was charged:  ['192', '158', '66', '177']
Whether is accused is convicted or acquited:  Convited
Date of verdict:  07/07/2014
